<a href="https://colab.research.google.com/github/kjboost/lecture-ai-engineering/blob/master/day3_homework_250511_06_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第二次トランプ政権の関税（表データ）に関するRAGの効果検証

このノートブックでは、Wikipediaの「第二次トランプ政権の関税」ページに含まれる特定の表データ（「表. 国別の「相互関税」の一覧（2025年4月2日時点）」）を用いて、Retrieval Augmented Generation (RAG) がLLMの回答精度に与える効果を検証します。特に、表データに含まれる具体的な数値情報（日本への関税率24%）について、LLMが単体では回答できないことを想定し、RAGによる情報提供の効果を確認します。

使用モデル: google/gemma-2-2b-jpn-it
参照資料: Wikipedia「第二次トランプ政権の関税」ページ中の「表.国別の「相互関税」の一覧（2025年4月2日時点）」

## 演習の方針

1.  **環境準備:** 必要なライブラリのインストールとモデルのロードを行います。
2.  **参照資料の準備:** Wikipediaページから特定の表データを取得し、RAGに使用できる形式に処理します。
3.  **ベースラインモデル評価:** 参照資料なしでLLMに質問し、回答を生成・評価します。
4.  **RAG実装と評価:** 参照資料（表データ）を組み込んだRAGでLLMに質問し、回答を生成・評価します。
5.  **結果分析と考察:** ベースラインとRAGありの回答を比較し、RAGの効果について考察します。

## 扱う質問と参照資料

以下の質問と、Wikipedia記事中の「表.国別の「相互関税」の一覧（2025年4月2日時点）」の内容をRAGの参照資料として使用します。特に、日本への関税率に関する質問は、LLMが単体では正確に回答できないことを想定しています。

参照元記事：["https://ja.wikipedia.org/wiki/%E7%AC%AC2%E6%AC%A1%E3%83%88%E3%83%A9%E3%83%B3%E3%83%97%E6%94%BF%E6%A8%A9%E3%81%AE%E9%96%A2%E7%A8%8E#cite_note-18"]
RAG参照資料：上記記事中の「表. 国別の「相互関税」の一覧（2025年4月2日時点）」のデータ

**質問リスト:**
1.  日本に課される「相互関税」の税率は何%ですか？
2.  記載されている国の中で、最も高い「相互関税」が課されているのはどの国ですか？またその税率は何%ですか？
3.  「相互関税」が10%とされている国はどこですか？
4.  このデータの時点は何時ですか？
5.  第二次トランプ政権で導入が検討されているとされる「普遍的な基礎関税」とは、どのような構想ですか？（※この質問に対するRAGは、表データのみを基に行われるため、適切な回答が得られない可能性があります。これはRAGの限界を示す例となります。）

**RAG参照資料の概要（表1のデータそのものを使用）:**
RAGでは、Wikipediaから抽出した「表. 国別の「相互関税」の一覧（2025年4月2日時点）」のテキストデータをそのまま参照情報として使用します。LLMはこの表データをコンテキストとして受け取り、質問に回答します。

## 演習環境の準備
必要なライブラリをインストールします。`pandas`は表の読み込みに使用します。

In [ ]:
!pip install --upgrade transformers
!pip install bitsandbytes accelerate
!pip install langchain-community #wikipedia # WikipediaLoaderは今回は使用しませんが、以前のコードとの整合性のため残します
!pip install chromadb # ベクトルストア用
!pip install sentence-transformers # エンベディングモデル用
!pip install openai # 評価用 (OpenAI APIキーが必要です)
!pip install pandas # 表の読み込み用
!pip install requests beautifulsoup4 # HTML取得・解析用

Hugging Face Hubにログインします（モデルのダウンロードに必要です）。ColabのSecretsに`hf_TOKEN`としてAPIトークンを保存しておくことを推奨します。

In [ ]:
from google.colab import userdata
import os

# Colab SecretsからHugging Faceトークンを取得
HUGGINGFACE_HUB_TOKEN = userdata.get('HF_TOKEN')

# 環境変数に設定
os.environ["HF_TOKEN"] = HUGGINGFACE_HUB_TOKEN

In [ ]:
# Hugging Face Hubにログイン
from huggingface_hub import login
try:
    login(HUGGINGFACE_HUB_TOKEN)
    print("Hugging Face Hubにログインしました。")
except Exception as e:
    print(f"Hugging Face Hubへのログインに失敗しました: {e}")
    print("Colab Secretsに 'hf_TOKEN' が正しく設定されているか確認してください。")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Hugging Face Hubにログインしました。


OpenAI APIキーも評価用に取得 (必要に応じて)

In [ ]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

使用するデバイスを設定します。

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用デバイス: {device}")

使用デバイス: cuda


乱数シードを設定します。

In [ ]:
import random
random.seed(42) # 再現性のためにシードを固定

LLMモデル (`google/gemma-2-2b-jpn-it`) を読み込みます。Colab無料版のT4 GPUで実行できるよう、4bit量子化を使用します。

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [ ]:
# 4bit量子化設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16, # T4 GPUはfloat16をサポート
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# モデルの読み込み
try:
    model = AutoModelForCausalLM.from_pretrained(
                model_name,
                device_map="auto", # 可能な限りGPUを使用
                quantization_config=bnb_config,
                torch_dtype=torch.bfloat16, # bfloat16を使用
            )
    print(f"モデル {model_name} をロードしました。")
except Exception as e:
    print(f"モデルのロードに失敗しました: {e}")
    print("GPUが利用可能か、または必要なライブラリが正しくインストールされているか確認してください。")

config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

モデル google/gemma-2-2b-jpn-it をロードしました。


LLMで回答を生成するための関数を定義します。

In [ ]:
def generate_output(query, model, tokenizer):
    #"""
    #指定されたモデルとトークナイザーを使用してクエリに対する回答を生成します。

    #Args:
    #    query (str): 生成するクエリまたはプロンプト。
    #    model: ロードされたLLMモデル。
    #    tokenizer: ロードされたトークナイザー。

    #Returns:
    #    str: 生成された回答テキスト。
    #"""
    messages = [
        {"role": "user", "content": query},
    ]

    # チャットテンプレートを適用し、入力IDに変換
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True, # モデルによっては不要な場合あり
        return_tensors="pt"
    ).to(model.device) # モデルと同じデバイスに配置

    # 生成停止条件 (EOSトークンや特定の特殊トークン)
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>") # GemmaのEOTトークン
    ]

    # 回答生成
    outputs = model.generate(
        input_ids,
        max_new_tokens=512, # 生成する最大トークン数（必要に応じて調整）
        eos_token_id=terminators,
        do_sample=False, # 決定的な生成 (Trueにすると多様な回答になる)
        # temperature=0.6, # do_sample=Trueの場合に使用
        # top_p=0.9,  # do_sample=Trueの場合に使用
    )

    # 生成されたトークンからテキストをデコード
    # 入力プロンプト部分を除外して回答のみを取得
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

評価関数を定義します。これはOpenAI APIを使用するため、実行にはAPIキーが必要です。

In [ ]:
# @title 評価実装 (OpenAI APIを使用)
# 注意: このセルを実行するには、Google Colab Secretsに 'OPENAI_API_KEY' が設定されている必要があります。
# また、OpenAI APIの利用料金が発生する可能性があります。

from openai import OpenAI
from google.colab import userdata # 既に上でインポート済み

# OpenAIクライアントの初期化
try:
    client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)
    print("OpenAIクライアントを初期化しました。")
except Exception as e:
    print(f"OpenAIクライアントの初期化に失敗しました: {e}")
    print("評価を実行するには、Colab Secretsに 'OPENAI_API_KEY' が正しく設定されているか確認してください。")
    client = None # 初期化失敗時はNoneとする



OpenAIクライアントを初期化しました。


In [ ]:
# 評価用のLLM生成関数 (OpenAI APIを使用)
def openai_generator(query):
    if client is None:
        print("OpenAIクライアントが初期化されていません。評価をスキップします。")
        return "評価スキップ"
    try:
        messages = [
                            {
                                "role": "user",
                                "content": query
                            }
                        ]

        response = client.chat.completions.create(
            model="gpt-4o-mini", # または他の適切なモデル
            messages=messages,
            max_tokens=256, # 生成する最大トークン数
            temperature=0 # 決定的な評価のために温度を0に設定
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"OpenAI API呼び出し中にエラーが発生しました: {e}")
        return "評価エラー"


# 評価指標関数 (Answer Accuracy)
def evaluate_answer_accuracy(query, response, reference):
    if client is None:
        return 0.0 # OpenAIクライアントがなければ評価できない

    # RagasのAnswer Accuracy評価プロンプトを参考に作成
    template_accuracy = (\
          "Instruction: You are a world class state of the art assistant for rating "
          "a User Answer given a Question and a Reference Answer. The Reference Answer is a perfect answer to the Question.\\n"
          "Rate the User Answer based on how consistent it is with the Reference Answer.\\n"
          "Say 4, if User Answer is full consistent and equivalent to Reference Answer "
          "in all terms, topics, numbers, metrics, dates and units.\\n"
          "Say 2, if User Answer is partially consistent and almost equivalent to Reference Answer "
          "in all terms, topics, numbers, metrics, dates and units.\\n"
          "Say 0, if User Answer is not consistent with Reference Answer or not accurate in all terms, topics,"
          "numbers, metrics, dates and units or the User Answer do not answer the question.\\n"
          "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\\n"
          "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\\n"
          "### Question: {query}\\n"
          "### User Answer: {user_answer}\\n"
          "### Reference Answer: {reference_answer}\\n"
          "The rating is:\\n"
      )

    # 評価プロンプトの生成
    eval_prompt = template_accuracy.format(
        query=query,
        user_answer=response,
        reference_answer=reference
    )

    # OpenAIで評価を実行
    score_text = openai_generator(eval_prompt)

    # スコアのパース
    try:
        score = int(score_text.strip())
        # スコアを0-4から0-1に正規化 (Ragasのデフォルトスコア範囲に合わせる場合)
        # score = score / 4.0
        return score # 0, 2, 4 のまま返す場合
    except:
        print(f"評価スコアのパースに失敗しました: '{score_text}'")
        return 0.0 # パース失敗時は0とする

In [ ]:
# 評価用のゴールドアンサーを定義 (表データに基づいて手動で作成)

gold_answers = {
    "日本に課される「相互関税」の税率は何%ですか？": "表によると、日本に課される「相互関税」の税率は24%です。",
    "記載されている国の中で、最も高い「相互関税」が課されているのはどの国ですか？またその税率は何%ですか？": "表によると、最も高い「相互関税」が課されているのはレソトで、その税率は50%です。", # 修正
    "「相互関税」が10%とされている国はどこですか？": "表において、「相互関税」が10%とされているのはすべての国と地域です。", # 修正
    "このデータの時点は何時ですか？": "表のデータは2025年4月2日時点の情報です。",
    "第二次トランプ政権で導入が検討されているとされる「普遍的な基礎関税」とは、どのような構想ですか？": "普遍的な基礎関税とは、第二次トランプ政権が誕生した場合に、全ての輸入品に対して一律に関税を引き上げる構想です。国内産業の保護、貿易赤字の削減、公正な貿易条件の確立を目指しています。" # この回答は表データには含まれませんが、ベースラインとの比較のために残します。
}

In [ ]:
# # 評価を実行する例 (OpenAI APIキーが必要です)
question_to_evaluate = list(gold_answers.keys())[0] # 最初の質問を例として評価
base_response = generate_output(question_to_evaluate, model, tokenizer)
rag_response = "ここにRAGによる回答を生成した結果を入れます" # RAGパートで生成した回答
base_score = evaluate_answer_accuracy(question_to_evaluate, base_response, gold_answers[question_to_evaluate])
print(f"質問: {question_to_evaluate}")
print(f"ベースモデル回答評価スコア: {base_score}")
rag_score = evaluate_answer_accuracy(question_to_evaluate, rag_response, gold_answers[question_to_evaluate])
print(f"RAGモデル回答評価スコア: {rag_score}")

質問: 日本に課される「相互関税」の税率は何%ですか？
ベースモデル回答評価スコア: 0
RAGモデル回答評価スコア: 0


In [ ]:
print("評価関数はOpenAI APIを使用します。実行にはAPIキー設定と利用料金が発生する可能性があります。")
print("評価コードはコメントアウトされています。実行する場合はコメントアウトを解除し、APIキーを設定してください。")

評価関数はOpenAI APIを使用します。実行にはAPIキー設定と利用料金が発生する可能性があります。
評価コードはコメントアウトされています。実行する場合はコメントアウトを解除し、APIキーを設定してください。


## 参照資料の準備 (Wikipediaの表データ)

Wikipediaページから「表1. 国別の「相互関税」の一覧（2025年4月2日時点）」を取得し、RAGに適した形に処理します。`pandas`を使用して表を読み込み、テキスト形式に変換します。

In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema import Document # LangChainのDocumentオブジェクトを使用
import requests  # requests libraryをインポート
from io import StringIO # StringIOをインポート
from bs4 import BeautifulSoup # BeautifulSoupをインポート

In [ ]:
# URL文字列の修正: Markdownリンク形式ではなく、純粋なURL文字列にする
wikipedia_url = "https://ja.wikipedia.org/wiki/%E7%AC%AC2%E6%AC%A1%E3%83%88%E3%83%A9%E3%83%B3%E3%83%97%E6%94%BF%E6%A8%A9%E3%81%AE%E9%96%A2%E7%A8%8E#cite_note-18"
table_title_identifier = "表1. 国別の「相互関税」の一覧" # 表を特定するためのキーワード（列名チェックが主になります）

print(f"Wikipediaページから表データをロードします: {wikipedia_url}")

table_documents = [] # 表データを格納するDocumentオブジェクトのリスト

try:
    # requestsとBeautifulSoupを使用してHTMLを取得・解析
    response = requests.get(wikipedia_url)
    response.raise_for_status() # HTTPエラーがあれば例外を発生
    response.encoding = 'utf-8' # エンコーディングを指定
    soup = BeautifulSoup(response.text, 'html.parser')

    # ページ内の全ての表要素を探す
    # pandas.read_htmlに直接soupオブジェクトを渡すと、BeautifulSoupでパース済みのHTMLから表を読み込める
    dfs = pd.read_html(StringIO(str(soup))) # soupオブジェクトを文字列化してStringIOでラップ

    print(f"ページから {len(dfs)} 個の表を検出しました。")

    # 目的の表（カラム名に「国または地域」と「関税率」が含まれるもの）を探す
    target_df = None
    target_table_element = None # 対象の表のBeautifulSoup要素を保持するための変数
    for i, df in enumerate(dfs):
        # DataFrameの列名を確認して目的の表を特定
        # 実際の表の列名に合わせてキーを調整してください
        if "国または地域" in df.columns and "関税率" in df.columns:
             target_df = df
             # pandas.read_htmlはDataFrameを返すため、元のBeautifulSoup要素を別途見つける必要がある
             # ここでは、検出したDataFrameに対応する元の<table>要素を探す
             # これは簡易的な方法であり、より頑健な方法が必要な場合もある
             tables_in_soup = soup.find_all('table')
             if i < len(tables_in_soup):
                 target_table_element = tables_in_soup[i]
             print(f"目的の表 (表 {i+1}) を検出しました。列名: {list(df.columns)}")
             break

    if target_df is not None:
        # 検出したDataFrameを整形された文字列に変換
        # index=FalseでDataFrameの行インデックスを非表示に
        # header=Trueで列ヘッダーを含める
        table_string = target_df.to_string(index=False, header=True)

        # 変換した表の文字列を表示
        print("\n--- 抽出した表データ (文字列形式) ---")
        print(table_string)
        print("\n-------------------------------------")

        # 抽出した表の文字列をLangChainのDocumentオブジェクトに変換
        # メタデータとしてソースURLや表のタイトルを含める
        table_document = Document(
            page_content=table_string,
            metadata={
                "source": wikipedia_url,
                "table_title": "国別の「相互関税」の一覧", # 検出した表のタイトルをメタデータに設定
                "description": "Wikipediaページの「表1. 国別の「相互関税」の一覧」のデータ本体"
            }
        )
        table_documents.append(table_document)

        # 表のキャプションまたは周囲のテキストを抽出
        additional_info = ""
        if target_table_element:
            # キャプションタグを探す
            caption = target_table_element.find('caption')
            if caption:
                additional_info += caption.get_text(strip=True) + "\n"
                print(f"\n--- 抽出したキャプション ---")
                print(caption.get_text(strip=True))
                print("----------------------------")
            else:
                 # キャプションがない場合、表の直前の要素からテキストを抽出する試み
                 # これはページの構造に依存するため、調整が必要な場合がある
                 previous_sibling = target_table_element.find_previous_sibling()
                 if previous_sibling and previous_sibling.name in ['h2', 'h3', 'p']: # 見出しや段落など
                     additional_info += previous_sibling.get_text(strip=True) + "\n"
                     print(f"\n--- 抽出した直前テキスト ({previous_sibling.name}) ---")
                     print(previous_sibling.get_text(strip=True))
                     print("------------------------------------")


        # 抽出した追加情報をDocumentオブジェクトとして追加
        if additional_info:
            additional_document = Document(
                page_content=additional_info.strip(), # 前後の空白を削除
                metadata={
                    "source": wikipedia_url,
                    "description": "Wikipediaページの表に関連するキャプションまたは周囲のテキスト"
                }
            )
            table_documents.append(additional_document)
            print(f"追加情報を含むドキュメントを生成しました。")


        # テキスト分割 (チャンク化)
        # 表データ本体と追加情報をまとめてチャンク化する
        # chunk_sizeは表全体と追加情報が収まるように十分大きく設定
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0) # 表全体が収まるようにチャンクサイズを大きめに
        chunked_documents = text_splitter.split_documents(table_documents)


        print(f"分割後のチャンク数: {len(chunked_documents)}")
        if chunked_documents:
            print("\n--- 分割されたチャンクの例 ---")
            # 複数のドキュメントがある場合、最初のチャンクが表データ本体とは限らない
            # ここではシンプルに最初のチャンクを表示
            print(chunked_documents[0].page_content)
            print("------------------------------")


        # エンベディングモデルのロード
        print("\nエンベディングモデルをロードします...")
        try:
            # オリジナルノートブックと同じモデルを使用
            # trust_remote_code=True を削除または False に変更
            embedding_model = SentenceTransformerEmbeddings(model_name="infly/inf-retriever-v1-1.5b")
            print("エンベディングモデルをロードしました。")
        except Exception as e:
            print(f"エンベディングモデルのロードに失敗しました: {e}")
            embedding_model = None # ロード失敗時はNoneとする

        # ベクトルストアの作成とデータの格納
        if embedding_model and chunked_documents:
            print("\nベクトルストアを作成し、チャンクを格納します...")
            try:
                # インメモリのChromaベクトルストアを使用
                vectorstore = Chroma.from_documents(documents=chunked_documents, embedding=embedding_model)
                print("ベクトルストアにチャンクを格納しました。")
                # retrieverはベクトルストアから関連性の高い上位N件を取得する役割
                # k=3は取得するチャンク数。表全体が1チャンクならk=1でも良いが、念のため複数取得できるようにしておく。
                retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # 表データから関連性の高い上位3件を取得
                print("リトリーバーを作成しました。")
            except Exception as e:
                 print(f"ベクトルストアの作成またはデータ格納に失敗しました: {e}")
                 vectorstore = None
                 retriever = None
        # 修正: 論理構造を修正し、不要なelseブロックを削除
        elif not embedding_model:
             print("エンベディングモデルがロードできなかったため、ベクトルストアとリトリーバーは作成されません。")
        elif not chunked_documents:
             print("チャンク化されたドキュメントがないため、ベクトルストアとリトリーバーは作成されません。")


    else: # target_df が None の場合
        print(f"指定された条件（列名に「国または地域」と「関税率」を含む）に合う表が見つかりませんでした。")
        chunked_documents = []
        vectorstore = None
        retriever = None
        embedding_model = None


except requests.exceptions.RequestException as e:
    print(f"ウェブページの取得中にエラーが発生しました: {e}")
    chunked_documents = []
    vectorstore = None
    retriever = None
    embedding_model = None
except Exception as e:
    print(f"表データの処理中にエラーが発生しました: {e}")
    chunked_documents = []
    vectorstore = None
    retriever = None
    embedding_model = None

Wikipediaページから表データをロードします: https://ja.wikipedia.org/wiki/%E7%AC%AC2%E6%AC%A1%E3%83%88%E3%83%A9%E3%83%B3%E3%83%97%E6%94%BF%E6%A8%A9%E3%81%AE%E9%96%A2%E7%A8%8E#cite_note-18
ページから 6 個の表を検出しました。
目的の表 (表 2) を検出しました。列名: ['国または地域', '関税率']

--- 抽出した表データ (文字列形式) ---
                    国または地域 関税率
                    アルジェリア 30%
                      アンゴラ 32%
                   バングラデシュ 37%
              ボスニア・ヘルツェゴビナ 35%
                      ボツワナ 37%
                      ブルネイ 24%
                     カンボジア 49%
                     カメルーン 11%
                       チャド 13%
                        中国 34%
                  コンゴ民主共和国 11%
                     赤道ギニア 13%
                      欧州連合 20%
フォークランド諸島 (United Kingdom) 41%
                      フィジー 32%
                      ガイアナ 38%
                       インド 26%
                    インドネシア 32%
                       イラク 39%
                     イスラエル 17%
                  コートジボワール 21%
                        日本 24%
                      ヨルダン 20

<ipython-input-18-195cd1e22a5f>:117: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="infly/inf-retriever-v1-1.5b")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

エンベディングモデルをロードしました。

ベクトルストアを作成し、チャンクを格納します...
ベクトルストアにチャンクを格納しました。
リトリーバーを作成しました。


## 3. ベースラインモデル評価 (RAGなし)

In [ ]:
# 評価用の質問リスト (表データに関する質問が中心)
questions = [
    "日本に課される「相互関税」の税率は何%ですか？",
    "記載されている国の中で、最も高い「相互関税」が課されているのはどの国ですか？またその税率は何%ですか？",
    "「相互関税」が10%とされている国はどこですか？",
    "このデータの時点は何時ですか？",
    "第二次トランプ政権で導入が検討されているとされる「普遍的な基礎関税」とは、どのような構想ですか？" # 表データにはないが、比較のために残す
]

# ベースライン回答を格納する辞書
baseline_responses = {}

print("--- ベースラインモデルによる回答生成 (RAGなし) ---")
for i, question in enumerate(questions):
    print(f"\n質問 {i+1}: {question}")
    try:
        # 表データに関する質問であることを明示的に伝えるプロンプトは使用しない
        # LLMの一般的な知識で回答できるかを確認
        response = generate_output(question, model, tokenizer)
        baseline_responses[question] = response
        print(f"回答:\n{response}")
    except Exception as e:
        print(f"回答生成中にエラーが発生しました: {e}")
        baseline_responses[question] = f"エラー: {e}"
# 評価用の質問リスト (表データに関する質問が中心)
questions = [
    "日本に課される「相互関税」の税率は何%ですか？",
    "記載されている国の中で、最も高い「相互関税」が課されているのはどの国ですか？またその税率は何%ですか？",
    "「相互関税」が10%とされている国はどこですか？",
    "このデータの時点は何時ですか？",
    "第二次トランプ政権で導入が検討されているとされる「普遍的な基礎関税」とは、どのような構想ですか？" # 表データにはないが、比較のために残す
]


--- ベースラインモデルによる回答生成 (RAGなし) ---

質問 1: 日本に課される「相互関税」の税率は何%ですか？
回答:
 日本に課される相互関税の税率は、**商品やサービスによって異なります**。 

一般的に、相互関税は**輸入品に対して課される税率**です。 

詳細な税率は、**輸入品の種類や国**によって異なります。 



 




質問 2: 記載されている国の中で、最も高い「相互関税」が課されているのはどの国ですか？またその税率は何%ですか？
回答:
申し訳ありませんが、私はリアルタイムの情報にアクセスできません。そのため、現在の相互関税の税率や国を特定することはできません。

相互関税に関する最新の情報を取得するには、以下の機関をご確認ください。

* **国際機関:**
    * 世界貿易機関 (WTO): https://www.wto.org/
    * 国際通貨基金 (IMF): https://www.imf.org/
* **政府機関:**
    * 各国の税務当局
    * 各国の貿易省


 




質問 3: 「相互関税」が10%とされている国はどこですか？
回答:
相互関税が10%とされている国は、**特定の国々**です。 

**具体的な国名**を挙げると、**正確な情報を得ることが難しい**場合があります。

**なぜなら**

* **相互関税の適用状況は常に変化する**
* **複数の国が相互関税を適用する場合もある**



 
 




質問 4: このデータの時点は何時ですか？
回答:
すみません、データの時点を教えていただけますか？ 


 




質問 5: 第二次トランプ政権で導入が検討されているとされる「普遍的な基礎関税」とは、どのような構想ですか？
回答:
##  第二次トランプ政権で導入が検討されていた「普遍的な基礎関税」とは？

「普遍的な基礎関税」は、トランプ政権において、**関税の導入を検討**していた政策の一環です。 

**基本的な構想**

* **消費税の導入**:  消費税を導入し、国民の消費活動に税金を課す。
* **公平性と透明性**:  税収の公平性と透明性を確保し、国民の理解を得る。
* **経済活性化**:  経済活動の活性化と雇用創出を促進する。

In [ ]:
# # ベースライン回答の評価 (OpenAI APIキーが必要です)
print("\n--- ベースライン回答の評価 ---")
if client is not None and gold_answers:
    baseline_scores = {}
    for question in questions:
        if question in gold_answers and question in baseline_responses:
            print(f"\n質問: {question}")
            score = evaluate_answer_accuracy(question, baseline_responses[question], gold_answers[question])
            baseline_scores[question] = score
            print(f"評価スコア: {score}")
        else:
            print(f"\n質問 '{question}' のゴールドアンサーまたはベースライン回答が見つかりません。評価をスキップします。")
else:
    print("\nOpenAIクライアントが初期化されていないか、ゴールドアンサーが定義されていません。ベースライン評価はスキップされます。")


--- ベースライン回答の評価 ---

質問: 日本に課される「相互関税」の税率は何%ですか？
評価スコア: 0

質問: 記載されている国の中で、最も高い「相互関税」が課されているのはどの国ですか？またその税率は何%ですか？
評価スコア: 0

質問: 「相互関税」が10%とされている国はどこですか？
評価スコア: 0

質問: このデータの時点は何時ですか？
評価スコア: 0

質問: 第二次トランプ政権で導入が検討されているとされる「普遍的な基礎関税」とは、どのような構想ですか？
評価スコア: 0


## 4. RAGを組み合わせた回答生成と評価
RAGを用いて関連情報（表データ）を取得し、その情報をLLMへの入力として組み込んで回答を生成・評価します。

In [ ]:
# RAG回答を格納する辞書
rag_responses = {}

print("\n--- RAGを組み合わせた回答生成 ---")
if retriever is not None and model is not None and tokenizer is not None:
    for i, question in enumerate(questions):
        print(f"\n質問 {i+1}: {question}")
        try:
            # RAGによる関連ドキュメント（表データチャンク）の取得
            # retrieverはベクトルストアから関連性の高いチャンクを取得します
            retrieved_docs = retriever.invoke(question)
            print(f"取得したドキュメント数: {len(retrieved_docs)}")
            # 取得したドキュメントの内容を表示 (確認用)
            # for j, doc in enumerate(retrieved_docs):
            #     print(f"--- 取得ドキュメント {j+1} ---")
            #     print(doc.page_content)
            #     print("-------------------------")


            # プロンプトの構築
            # 取得したドキュメント（表データチャンク）をコンテキストとしてプロンプトに含める
            context = "\n\n".join([doc.page_content for doc in retrieved_docs])
            # 質問自体に「表1によると」という記述は含めないが、RAGのプロンプトで表データであることを明示する
            rag_prompt = f"""以下の参考情報を考慮して、質問に回答してください。参考情報はWikipediaの表データです。

[参考情報（表データ）]
{context}

[質問]
{question}
"""
            # RAGプロンプトで回答生成
            response = generate_output(rag_prompt, model, tokenizer)
            rag_responses[question] = response
            print(f"回答:\n{response}")

        except Exception as e:
            print(f"RAG回答生成中にエラーが発生しました: {e}")
            rag_responses[question] = f"エラー: {e}"
else:
    print("リトリーバーまたはモデルが正しく初期化されていないため、RAG回答生成はスキップされます。")




--- RAGを組み合わせた回答生成 ---

質問 1: 日本に課される「相互関税」の税率は何%ですか？
取得したドキュメント数: 2
回答:
日本に課される「相互関税」の税率は24%です。




質問 2: 記載されている国の中で、最も高い「相互関税」が課されているのはどの国ですか？またその税率は何%ですか？
取得したドキュメント数: 2
回答:
最も高い「相互関税」が課されている国は **シリア** です。税率は **41%** です。 





質問 3: 「相互関税」が10%とされている国はどこですか？
取得したドキュメント数: 2
回答:
「相互関税」が10%とされている国は、**すべての国と地域**です。 





質問 4: このデータの時点は何時ですか？
取得したドキュメント数: 2
回答:
このデータの時点は2025年4月2日です。




質問 5: 第二次トランプ政権で導入が検討されているとされる「普遍的な基礎関税」とは、どのような構想ですか？
取得したドキュメント数: 2
回答:
参考情報には「普遍的な基礎関税」に関する情報は含まれていません。


 





In [ ]:
# # RAG回答の評価 (OpenAI APIキーが必要です)
print("\n--- RAG回答の評価 ---")
if client is not None and gold_answers:
    rag_scores = {}
    for question in questions:
        if question in gold_answers and question in rag_responses:
            print(f"\n質問: {question}")
            score = evaluate_answer_accuracy(question, rag_responses[question], gold_answers[question])
            rag_scores[question] = score
            print(f"評価スコア: {score}")
        else:
            print(f"\n質問 '{question}' のゴールドアンサーまたはRAG回答が見つかりません。評価をスキップします。")
else:
    print("\nOpenAIクライアントが初期化されていないか、ゴールドアンサーが定義されていません。RAG評価はスキップされます。")


--- RAG回答の評価 ---

質問: 日本に課される「相互関税」の税率は何%ですか？
評価スコア: 4

質問: 記載されている国の中で、最も高い「相互関税」が課されているのはどの国ですか？またその税率は何%ですか？
評価スコア: 0

質問: 「相互関税」が10%とされている国はどこですか？
評価スコア: 2

質問: このデータの時点は何時ですか？
評価スコア: 4

質問: 第二次トランプ政権で導入が検討されているとされる「普遍的な基礎関税」とは、どのような構想ですか？
評価スコア: 0
